In [3]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import re

import sys
sys.path.append('functions')
import preprocessing_fncs as ppf
import elastic_search_fncs as esf

# Connecting

In [4]:
# Details of the dataset
db_host = 'https://athena.london.gov.uk'
db_user = 'odbc_readonly'
db_pass = 'odbc_readonly'
db_port = '10099'
db_name = 'gla-ldd-external'

# Creates connection to the dataset
es = Elasticsearch(
    [f"{db_host}:{db_port}"],
    basic_auth=(db_user, db_pass),
    verify_certs=True,
    ca_certs='athena_es_full_chain.crt'
)

# Check connection
if es.ping():
    print("Connected to Elasticsearch!")
else:
    print("Could not connect to Elasticsearch.")

Connected to Elasticsearch!


# Get the data from 2015 to 2019

## Existing Residential Data

In [3]:
all_years_df = [] # save the data into this dataframe

for year in range(2015, 2020):  # 2015–2019
    query = {
        "query": {
            "bool": {
                # conditions that must be met
                "must": [ 
                    {
                        "range": {
                            # valid data between 2015-019
                            "valid_date": {
                                "gte": f"01/01/{year}",
                                "lt": f"01/01/{year + 1}"
                            }
                        }
                    }
                ],
                # The conditions that should be met
                "should": [
                    {
                        "range": {
                            "application_details.residential_details.total_no_existing_residential_units": {
                                "gte": 1
                            }
                        }
                    },
                    {
                        "range": {
                            "application_details.residential_details.total_no_proposed_residential_units": {
                                "gte": 1
                            }
                        }
                    }
                ],
                "minimum_should_match": 1 # At least meet one of the condition
            }
        },
        "_source": [
            "valid_date",
            "borough",
            "application_details.residential_details.total_no_existing_residential_units",
            "application_details.residential_details.total_no_proposed_residential_units",
            "street_name",
            "site_name",
            "polygon", 
            "wgs84_polygon", # geo
            "description" # main target
        ]
    }
    
    # Elasticsearch query
    response = es.search(index="applications", body=query, scroll="2m", size=10000)
    scroll_id = response['_scroll_id']
    hits = response['hits']['hits']

    all_hits = []
    all_hits.extend(hits)

    while len(hits) > 0:
        response = es.scroll(scroll_id=scroll_id, scroll="2m")
        scroll_id = response['_scroll_id']
        hits = response['hits']['hits']
        all_hits.extend(hits)

    df_raw = pd.json_normalize(all_hits)
    df_cleaned = ppf.format_df(df_raw)
    df_cleaned['year'] = year

    all_years_df.append(df_cleaned)

# combined all the data
df_london_all = pd.concat(all_years_df, ignore_index=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_12100\2052650244.py:53: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="applications", body=query, scroll="2m", size=10000)
C:\Users\hp\AppData\Local\Temp\ipykernel_12100\2052650244.py:53: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="applications", body=query, scroll="2m", size=10000)
C:\Users\hp\AppData\Local\Temp\ipykernel_12100\2052650244.py:53: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific paramete

ConnectionTimeout: Connection timed out

In [ ]:
print(df_london_all.shape)
print(df_london_all.head())

## Proposed Residential Data

In [ ]:
all_years_df2 = [] # save the data into this dataframe

for year in range(2015, 2020):  # 2015–2019
    query = {
        "query": {
            "bool": {
                # conditions that must be met
                "must": [ 
                    {
                        "range": {
                            # desition data between 2015-019
                            "decision_date": {
                                "gte": f"01/01/{year}",
                                "lt": f"01/01/{year + 1}"
                            }
                        }
                    }
                ],
                # The conditions that should be met
                "should": [
                    {
                        "range": {
                            "application_details.residential_details.total_no_existing_residential_units": {
                                "gte": 1
                            }
                        }
                    },
                    {
                        "range": {
                            "application_details.residential_details.total_no_proposed_residential_units": {
                                "gte": 1
                            }
                        }
                    }
                ],
                "minimum_should_match": 1 # At least meet one of the condition
            }
        },
        "_source": [
            "decision_date",
            "borough",
            "application_details.residential_details.total_no_existing_residential_units",
            "application_details.residential_details.total_no_proposed_residential_units",
            "street_name",
            "site_name",
            "polygon", 
            "wgs84_polygon", # geo
            "description" # main target
        ]
    }
    
    # Elasticsearch query
    response = es.search(index="applications", body=query, scroll="2m", size=10000)
    scroll_id = response['_scroll_id']
    hits = response['hits']['hits']

    all_hits = []
    all_hits.extend(hits)

    while len(hits) > 0:
        response = es.scroll(scroll_id=scroll_id, scroll="2m")
        scroll_id = response['_scroll_id']
        hits = response['hits']['hits']
        all_hits.extend(hits)

    df_raw = pd.json_normalize(all_hits)
    df_cleaned = ppf.format_df(df_raw)
    df_cleaned['year'] = year

    all_years_df2.append(df_cleaned)

# combined all the data
df_london_all2 = pd.concat(all_years_df2, ignore_index=True)

In [ ]:
print(df_london_all2.shape)

There is a large gap between the decision date and the valid date.

1. change into decision date?

2. stick to valid date but change the range to a longer time period?

## All applications between 2010 - 2015

In [5]:
all_years_df3 = []  # 用于保存所有年份的数据

for year in range(2014, 2015):  # 实际是2010–2014
    query = {
        "query": {
            "bool": {
                "must": [
                    {
                        "range": {
                            "valid_date": {
                                "gte": f"01/01/{year}",
                                "lt": f"01/01/{year + 1}"
                            }
                        }
                    }
                ],
            }
        },
        "_source": [
            "valid_date",
            "decision_date",
            "borough",
            "application_details.residential_details.total_no_existing_residential_units",
            "application_details.residential_details.total_no_proposed_residential_units",
            "street_name",
            "site_name",
            "polygon",
            "wgs84_polygon",
            "description"
        ]
    }

    try:
        # 初始化查询
        response = es.search(index="applications", body=query, scroll="2m", size=10000)
        scroll_id = response.get('_scroll_id')
        hits = response['hits']['hits']
        all_hits = hits.copy()

        # 开始滚动
        while hits:
            try:
                response = es.scroll(scroll_id=scroll_id, scroll="2m")
                scroll_id = response.get('_scroll_id')
                hits = response['hits']['hits']
                all_hits.extend(hits)
            except Exception as scroll_err:
                print(f"[{year}] Scroll failed: {scroll_err}")
                break

        # 格式化 DataFrame
        if all_hits:
            df_raw = pd.json_normalize(all_hits)
            df_cleaned = ppf.format_df(df_raw)
            df_cleaned['year'] = year
            all_years_df3.append(df_cleaned)
            print(f"[{year}] Completed with {len(df_cleaned)} records.")
        else:
            print(f"[{year}] No results returned.")

    except Exception as e:
        print(f"[{year}] Initial query failed: {e}")
        continue

# 合并所有年份的数据
if all_years_df3:
    df_london_all3 = pd.concat(all_years_df3, ignore_index=True)
    print(f"Total records combined: {len(df_london_all3)}")
else:
    print("No data retrieved.")

df_london_all3 = pd.concat(all_years_df3, ignore_index=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_25444\2606121568.py:35: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="applications", body=query, scroll="2m", size=10000)


[2014] Scroll failed: NotFoundError(404, 'search_phase_execution_exception', 'No search context found for id [10730912]', No search context found for id [10730912])
[2014] Completed with 10000 records.
Total records combined: 10000


In [6]:
print(df_london_all3.shape)

(10000, 16)


In [7]:
df_london_all3.to_csv("data/cleaned_projects_raw_14.csv", index=False)

# Data Cleaning

- Select the text (description column, delete NA lines)
- Clean the text (excessive spaces and special characters)
- Split the descriptions into sentences
- Vectorizes sentences using SBERT

In [ ]:
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import joblib
import os
import nltk
# nltk.download('punkt')      # Normal Sentence Segmentation Model
# nltk.download('punkt_tab')  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
# reconfirm the columns
df_london_all = pd.read_csv("data\cleaned_projects_raw.csv")

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\hp\AppData\Local\Temp\ipykernel_2108\2394944548.py:2: SyntaxWarning: invalid escape sequence '\c'
  df_london_all = pd.read_csv("data\cleaned_projects_raw.csv")


In [5]:
# set a new copy
df = df_london_all.copy()
# keep only the non-empty text
df = df[df['description'].notna()]

In [6]:
# excessive spaces and special characters
df['description'] = df['description'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [7]:
# split
df['sentences'] = df['description'].apply(sent_tokenize)

In [8]:
# flatten all the sentences
all_sentences = df['sentences'].explode().tolist()

In [9]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

！This took too much time (around 40 mins), consider moving to google colab.

Here used paraphrase-MiniLM-L6-v2 to minimize the time, can change into 768 vector in colab.

In [10]:
# encode all sentences (with caching)
BATCH_SIZE = 64
CACHE_FILE = 'sbert_vectors.pkl'

if os.path.exists(CACHE_FILE):
    print("Loading cached embeddings...")
    sentence_embeddings = joblib.load(CACHE_FILE)
else:
    print("Encoding...")
    sentence_embeddings = model.encode(
        all_sentences,
        batch_size=BATCH_SIZE,
        show_progress_bar=True
    )
    joblib.dump(sentence_embeddings, CACHE_FILE)
    print("Embeddings cached!")

Loading cached embeddings...


In [11]:
emb_idx = 0
final_vectors = []

for sent_list in tqdm(df['sentences']):
    count = len(sent_list)
    if count == 0:
        final_vectors.append(np.zeros(model.get_sentence_embedding_dimension()))
    else:
        vecs = sentence_embeddings[emb_idx: emb_idx + count]
        final_vectors.append(np.mean(vecs, axis=0))
        emb_idx += count

100%|██████████| 188278/188278 [00:01<00:00, 121085.70it/s]


In [12]:
# DataFrame + merge back to main list
vec_df = pd.DataFrame(final_vectors)
vec_df.columns = [f'sbert_{i}' for i in range(vec_df.shape[1])]
df_vectors = pd.concat([df.reset_index(drop=True), vec_df], axis=1)

In [13]:
# save a copy
df_vectors.to_parquet('sbert_encoded_data.parquet')

In [15]:
print(df_vectors.shape)

(188278, 401)
